## 如何查看 Agent 的详细思维过程

### 1. 设置环境变量
```python
os.environ['LANGCHAIN_VERBOSE'] = 'true'
```

### 2. Agent 已经默认设置了 verbose=True
在 `react_agent.py` 的第 438 行：
```python
executor_config = {
    "agent": agent,
    "tools": tools,
    "verbose": True,  # 这里已经启用了详细输出
    "max_iterations": 50,
    "handle_parsing_errors": True
}
```

### 3. 当前输出显示的内容
- **工具调用决策**：显示 Agent 决定调用哪个工具及参数
- **工具执行结果**：显示每个工具的返回值
- **执行步骤**：显示 Agent 的执行流程

### 4. LangChain 的限制
- Agent 的内部"思考"（Chain of Thought）不会完全暴露
- 只能看到工具调用的决策和结果
- 这是 LangChain 设计的限制，不是配置问题

### 5. 如需更详细的输出
可以考虑：
- 修改 system prompt 让 Agent 解释其推理过程
- 使用 LangSmith 追踪（需要 API key）
- 添加自定义 callbacks 来拦截更多信息

## Agent Research Notebook - 使用 Moonshot (Kimi) API

本 notebook 展示如何使用 GenericReactAgent，并配置为使用 Moonshot (Kimi) API。

### API 配置信息
- **Base URL**: `https://api.moonshot.cn/v1`
- **Model**: `kimi-k2-0711-preview`（128k 上下文窗口）
- **API Key**: 需要设置环境变量 `MOONSHOT_API_KEY`

### 上下文窗口自动配置

系统会根据模型自动设置上下文窗口大小，并相应调整 memory 配置：

| 模型 | 上下文窗口 | 自动 memory 限制 |
|------|-----------|-----------------|
| kimi-k2-0711-preview | 128k | 102k (80%) |
| gpt-4-turbo | 128k | 102k (80%) |
| claude-3-opus | 200k | 160k (80%) |
| deepseek-chat | 32k | 25k (80%) |

### 设置 API Key 的方法

1. **使用 .env 文件（推荐）**：
   ```bash
   # 在项目根目录创建 .env 文件
   MOONSHOT_API_KEY=your_api_key_here
   ```

2. **使用环境变量**：
   ```bash
   export MOONSHOT_API_KEY=your_api_key_here
   ```

3. **在 notebook 中临时设置**：
   ```python
   import os
   os.environ['MOONSHOT_API_KEY'] = 'your_api_key_here'
   ```

### 其他支持的 LLM 配置示例

```python
# DeepSeek (默认)
config = ReactAgentConfig(
    work_dir="output",
    # 无需指定 LLM 参数，默认使用 DeepSeek
)

# OpenAI
config = ReactAgentConfig(
    work_dir="output",
    llm_model="gpt-4-turbo-preview",
    llm_base_url="https://api.openai.com/v1",
    llm_api_key_env="OPENAI_API_KEY"
)

# Claude (Anthropic)
config = ReactAgentConfig(
    work_dir="output",
    llm_model="claude-3-opus-20240229",
    llm_base_url="https://api.anthropic.com/v1",
    llm_api_key_env="ANTHROPIC_API_KEY"
)

# 手动指定上下文窗口
config = ReactAgentConfig(
    work_dir="output",
    llm_model="custom-model",
    llm_base_url="https://api.custom.com/v1",
    llm_api_key_env="CUSTOM_API_KEY",
    context_window=200000  # 手动指定 200k 上下文窗口
)
```

### Memory 级别详解

GenericReactAgent 支持三种记忆级别：

1. **NONE（无记忆）**
   - 每次对话都是独立的
   - 适合单次任务执行
   
2. **SMART（智能摘要缓冲）** - 使用 `ConversationSummaryBufferMemory`
   - 保留最近的对话原文直到达到 token 限制
   - 对超出限制的历史对话自动生成摘要
   - 适合长对话场景，既保留了重要信息又控制了 token 使用
   - 对于 Kimi K2 (128k)，可以保留约 100k tokens 的原始对话
   
3. **PRO（持久化存储）**
   - 使用 SQLite 数据库保存所有历史
   - 支持跨会话恢复对话
   - 适合需要长期记忆的项目

### ConversationSummaryBufferMemory 的优势

对于大上下文模型（如 Kimi K2 的 128k），使用 `ConversationSummaryBufferMemory` 的好处：

1. **智能管理历史**：自动决定哪些对话需要摘要，哪些保留原文
2. **最大化上下文利用**：充分利用 128k 的上下文窗口
3. **保留关键信息**：通过摘要保留早期对话的关键信息
4. **动态调整**：根据对话长度自动调整摘要策略

# 初始化

In [1]:

# 设置详细输出的环境变量示例
import os
import sys

# 保存原始代理设置
original_proxy = {
    'http_proxy': os.environ.get('http_proxy'),
    'https_proxy': os.environ.get('https_proxy'),
    'all_proxy': os.environ.get('all_proxy')
}

# 加载 .env 文件中的环境变量（如果使用 python-dotenv）
try:
    from dotenv import load_dotenv  # type: ignore
    load_dotenv()
except ImportError:
    print("提示：python-dotenv 未安装，跳过 .env 文件加载")
    print("可以使用 pip install python-dotenv 安装")

# 确保设置了 MOONSHOT_API_KEY
if not os.getenv('MOONSHOT_API_KEY'):
    print("警告：未设置 MOONSHOT_API_KEY 环境变量")
    print("请使用以下方式之一设置：")
    print("1. 创建 .env 文件并添加 MOONSHOT_API_KEY=your_api_key")
    print("2. 设置环境变量：export MOONSHOT_API_KEY=your_api_key")
    print("3. 在代码中设置：os.environ['MOONSHOT_API_KEY'] = 'your_api_key'")

# 方法1: 设置 LANGCHAIN_VERBOSE
# os.environ['LANGCHAIN_VERBOSE'] = 'true'

# 方法2: 设置日志级别
import logging
# logging.basicConfig(level=logging.DEBUG)
# logging.getLogger("langchain").setLevel(logging.DEBUG)

# 方法3: 使用 callbacks
try:
    from langchain_core.callbacks import StdOutCallbackHandler  # type: ignore
except ImportError:
    # 如果新的导入路径不存在，尝试旧路径
    try:
        from langchain.callbacks import StdOutCallbackHandler  # type: ignore
    except ImportError:
        print("提示：无法导入 StdOutCallbackHandler，某些功能可能不可用")

print("开始导入 react_agent...")
# 创建带 callback 的 agent
from react_agent import GenericReactAgent, ReactAgentConfig, MemoryLevel
print("导入完成!")

# 恢复代理设置（react_agent.py 可能会删除它们）
for key, value in original_proxy.items():
    if value:
        os.environ[key] = value
        
print("环境准备完成。")

开始导入 react_agent...
导入完成!
环境准备完成。


In [2]:
config = ReactAgentConfig(
    work_dir="/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need",
    memory_level=MemoryLevel.SMART,
    llm_model="kimi-k2-turbo-preview",
    llm_base_url="https://api.moonshot.cn/v1",
    llm_api_key_env="MOONSHOT_API_KEY",
    llm_temperature=0,
    # 从文件加载UML探索框架
    exploration_prompt_file="uml_exploration_prompt.md",
    enable_project_exploration=True,
    
    # context_window 会自动设置为 131072 (128k)
    # max_token_limit 会自动调整为 104857 (上下文窗口的 80%)
)

# config = ReactAgentConfig(
#     work_dir="./",
#     memory_level=MemoryLevel.SMART,
#     llm_model="deepseek-chat",
#     llm_base_url="https://api.deepseek.com/v1",
#     llm_api_key_env="DEEPSEEK_API_KEY",
#     llm_temperature=0
#     # context_window 会自动设置为 131072 (128k)
#     # max_token_limit 会自动调整为 104857 (上下文窗口的 80%)
# )

agent = GenericReactAgent(config)

# 打印配置信息
print(f"模型: {config.llm_model}")
print(f"上下文窗口: {config.context_window} tokens")
print(f"内存 token 限制: {config.max_token_limit}")

task='''
你好
'''

agent.execute_task(task)

模型: kimi-k2-turbo-preview
上下文窗口: 131072 tokens
内存 token 限制: 104857

[Agent-kimi-k2-turbo-preview] > Executing task...

👤 用户: 
你好


💬 工具结果 (None):
   你好！我是你的任务执行助手，随时准备帮助你完成各种任务。

我可以帮你：
- 分析项目结构和代码
- 搜索和修改文件
- 运行命令和测试
- 创建或更新文档
- 调试问题
- 执行各种开发任务

请告诉我你需要我帮你做什么？

[Agent-kimi-k2-turbo-preview] > Task completed.

=== [Agent-kimi-k2-turbo-preview] 最终结果 ===

你好！我是你的任务执行助手，随时准备帮助你完成各种任务。

我可以帮你：
- 分析项目结构和代码
- 搜索和修改文件
- 运行命令和测试
- 创建或更新文档
- 调试问题
- 执行各种开发任务

请告诉我你需要我帮你做什么？


'你好！我是你的任务执行助手，随时准备帮助你完成各种任务。\n\n我可以帮你：\n- 分析项目结构和代码\n- 搜索和修改文件\n- 运行命令和测试\n- 创建或更新文档\n- 调试问题\n- 执行各种开发任务\n\n请告诉我你需要我帮你做什么？'


💭 [记忆更新] # 知识库



In [ ]:
task='''
/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/.agents/Agent-kimi-k2-turbo-preview/long_term_data  
删除这个目录下所有文件

'''

x=agent.execute_task(task)
print(x)


[Agent-kimi-k2-turbo-preview] > Executing task...

👤 用户: 
/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/.agents/Agent-kimi-k2-turbo-preview/long_term_data  
删除这个目录下所有文件



🤔 [Agent-kimi-k2-turbo-preview] AI 思考: 需要使用工具来完成任务
🔧 调用工具: execute_command
   参数: {'command': 'rm -rf /home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/.agents/Agent-kimi-k2-turbo-preview/long_term_data/*', 'working_dir': '/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need'}

💬 工具结果 (execute_command):
   Exit code: 0


🤔 [Agent-kimi-k2-turbo-preview] AI 思考: 需要使用工具来完成任务
🔧 调用工具: execute_command
   参数: {'command': 'ls -la /home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need/.agents/Agent-kimi-k2-turbo-preview/long_term_data/', 'working_dir': '/home/guci/aiProjects/mda/pim-compiler/react_is_all_you_need'}

💬 工具结果 (execute_command):
   Exit code: 0
total 8
drwxrwxr-x 2 guci guci 4096  8月  5 19:59 .
drwxrwxr-x 4 guci guci 4096  8月  5 05:12 ..


💬 工具结果 (None):
   ✅ 完成！目录 `/home/g

   📊 分析项目概述...
   📊 分析Use Case视图...

💭 [记忆更新] # 知识库

   📊 分析Package视图...
   📊 分析Class视图...
   📊 分析Interaction视图...
   📊 生成综合分析...
✅ [项目探索] 完成！已生成完整的UML分析文档 (11452 字符)
   📄 包含：项目概述 + 4个UML视图 + 综合分析
💡 [项目理解] 探索完成，将在下次任务时更新模型


In [4]:
task='''
阐述系统架构设计
'''

x=agent.execute_task(task)
print(x)


🔄 [项目理解] 检测到新的项目理解，正在更新...

♻️ [知识热加载] 正在更新知识系统...
✅ [知识热加载] 完成！已加载 1 个知识源，对话历史已保留
💡 现在Agent基于最新的UML模型工作


[Agent-kimi-k2-turbo-preview] > Executing task...

👤 用户: 
阐述系统架构设计


💬 工具结果 (None):
   # 系统架构设计阐述

## 1. 架构概览

这是一个基于 **LangGraph** 的下一代 React Agent 框架，采用 **微内核 + 插件式多 Agent** 架构模式，实现"自然语言即代码"的 AI 编程与协作平台。

## 2. 分层架构

```
┌─────────────────────────────────────────┐
│           应用层 (examples)              │  ← 业务场景示例
├─────────────────────────────────────────┤
│           代理层 (.agents)               │  ← 独立Agent实例
│  ┌─────────────┬───────────┬─────────┐ │
│  │ 业务Agent   │ 调试Agent │ 测试Agent│ │
│  └─────────────┴───────────┴─────────┘ │
├─────────────────────────────────────────┤
│...
   [省略 1703 字符]

[Agent-kimi-k2-turbo-preview] > Task completed.

=== [Agent-kimi-k2-turbo-preview] 最终结果 ===

# 系统架构设计阐述

## 1. 架构概览

这是一个基于 **LangGraph** 的下一代 React Agent 框架，采用 **微内核 + 插件式多 Agent** 架构模式，实现"自然语言即代码"的 AI 编程与协作平台。

## 2. 分层架构

```
┌─────────────────────────────────────────┐
│          


💭 [记忆更新] # 知识库



In [5]:
task='''
把核心要素映射到计算机组成的核心要素
'''

x=agent.execute_task(task)
print(x)


[Agent-kimi-k2-turbo-preview] > Executing task...

👤 用户: 
把核心要素映射到计算机组成的核心要素


💬 工具结果 (None):
   # React Agent 框架 ↔ 计算机组成原理 映射关系

## 🧠 核心映射总览

```
React Is All You Need 框架    ↔    计算机组成原理
───────────────────────────────────────────────
Agent实例空间 (.agents/)      ↔    进程空间 (Process Space)
核心运行时 (core/)            ↔    操作系统内核 (Kernel)
记忆系统                     ↔    存储器层次结构 (Memory Hierarchy)
知识库                       ↔    指令集架构 (ISA)
工具链                       ↔    输入输出系统 (I/O System)
调试器                       ↔    调试监控器 (Debugger)
```

## 🔍 详细映射分析

### 1. Agent实例空间 ↔ 进程空间
| React Agent概念 | 计算机组...
   [省略 2415 字符]

[Agent-kimi-k2-turbo-preview] > Task completed.

=== [Agent-kimi-k2-turbo-preview] 最终结果 ===

# React Agent 框架 ↔ 计算机组成原理 映射关系

## 🧠 核心映射总览

```
React Is All You Need 框架    ↔    计算机组成原理
───────────────────────────────────────────────
Agent实例空间 (.agents/)      ↔    进程空间 (Process Space)
核心运行时 (core/)            ↔    操作系统内核 (Kernel)
记忆系统                     ↔    存储器层次结构 (Memory Hiera


💭 [记忆更新] # 知识库



In [ ]:
# 测试 ConversationSummaryBufferMemory 的效果
# 这个 memory 会智能管理对话历史

# 创建一个新的 agent 实例来测试长对话
test_config = ReactAgentConfig(
    work_dir="output/memory_test",
    memory_level=MemoryLevel.SMART,
    llm_model="kimi-k2-0711-preview",
    llm_base_url="https://api.moonshot.cn/v1",
    llm_api_key_env="MOONSHOT_API_KEY",
    llm_temperature=0
)

test_agent = GenericReactAgent(test_config)

# 模拟多轮对话
print("=== 测试 ConversationSummaryBufferMemory ===\n")

# 第一轮
test_agent.execute_task("我叫张三，我是一名软件工程师")

# 第二轮  
test_agent.execute_task("我最喜欢的编程语言是Python")

# 第三轮
test_agent.execute_task("我的名字是什么？我的职业是什么？")

# 随着对话继续，早期的对话会被自动摘要
# 但重要信息（如名字、职业）会被保留

In [ ]:
# 测试 LangChain 缓存效果
# temperature=0 时，相同的输入会使用缓存

import time

# 第一次调用（会调用 API）
start = time.time()
agent.execute_task("什么是 Python？")
first_time = time.time() - start
print(f"\n第一次调用耗时: {first_time:.2f} 秒")

# 第二次调用相同问题（应该使用缓存）
start = time.time()
agent.execute_task("什么是 Python？")
second_time = time.time() - start
print(f"\n第二次调用耗时: {second_time:.2f} 秒")

print(f"\n缓存加速: {first_time/second_time:.1f}x")

In [ ]:
# 创建配置 - 使用 Moonshot API
config = ReactAgentConfig(
    work_dir="output",
    memory_level=MemoryLevel.NONE,
    # knowledge_file="knowledge/综合知识.md",
    llm_model="kimi-k2-0711-preview",
    llm_base_url="https://api.moonshot.cn/v1",
    llm_api_key_env="MOONSHOT_API_KEY",
    llm_temperature=0
)

# 创建 Agent
agent = GenericReactAgent(config)

# 执行任务
task="项目根目录是/home/guci/aiProjects/mda/pim-compiler，他的体系架构和核心组件是什么？"
# task="编程规范和项目结构知识是什么？"
agent.execute_task(task)

In [ ]:
# 创建配置 - 使用 Moonshot API
config = ReactAgentConfig(
    work_dir="output",
    memory_level=MemoryLevel.NONE,
    knowledge_file="pim_to_psm_knowledge.md",
    llm_model="kimi-k2-0711-preview",
    llm_base_url="https://api.moonshot.cn/v1",
    llm_api_key_env="MOONSHOT_API_KEY",
    llm_temperature=0
)

# 创建 Agent
psm_expert = GenericReactAgent(config)
psm_expert.specification="psm专家，擅长将pim文档转化成psm文档"

# 执行任务
psm_expert.execute_task("/home/guci/aiProjects/mda/pim-compiler/examples/blog.md  转化成psm保存在/home/guci/aiProjects/mda/pim-compiler/examples/blog_psm.md")

### 使用 Google 搜索功能

GenericReactAgent 现在支持 Google 搜索和网页读取功能。需要先设置 Google API：

1. 获取 Google API Key: https://console.cloud.google.com/apis/credentials
2. 创建自定义搜索引擎: https://programmablesearchengine.google.com/
3. 在 .env 文件中设置：
   ```
   GOOGLE_API_KEY=your_api_key_here
   GOOGLE_CSE_ID=your_cse_id_here
   ```

如果需要代理访问 Google，还需设置：
```
HTTP_PROXY=http://your_proxy:port
```

In [ ]:
# 测试搜索功能的 Agent
# 注意：需要先设置 Google API 凭证

# 创建带搜索功能的 Agent
search_config = ReactAgentConfig(
    work_dir="output/search_test",
    memory_level=MemoryLevel.SMART,
    llm_model="kimi-k2-0711-preview",
    llm_base_url="https://api.moonshot.cn/v1",
    llm_api_key_env="MOONSHOT_API_KEY",
    llm_temperature=0
)

search_agent = GenericReactAgent(search_config)

# 测试搜索功能
task = """
请帮我网络搜索关于 "LangChain memory 类型" 的信息，并总结不同类型的 memory 的特点。
"""

search_agent.execute_task(task)